In [1]:
push!(LOAD_PATH,".")

4-element Vector{String}:
 "@"
 "@v#.#"
 "@stdlib"
 "."

# Import CHAKRA (and an implementation)

In [2]:
using Chakra, ChakraImp

# Define an attribute type
The following defines a type pc of pitch classes and associates it an attribute name ":pc"

In [3]:
struct pc
    val::Char
end
Chakra.@associatedType(:pc,pc)

# Represent choral
The following represents the first 10 notes of a bach choral as a sequence of constituents with pitch class attributes.

In [4]:
e1 = set(delim(),:pc,pc('G'))
e2 = set(delim(),:pc,pc('G'))
e3 = set(delim(),:pc,pc('D'))
e4 = set(delim(),:pc,pc('B'))
e5 = set(delim(),:pc,pc('A'))
e6 = set(delim(),:pc,pc('G'))
e7 = set(delim(),:pc,pc('G'))
e8 = set(delim(),:pc,pc('A'))
e9 = set(delim(),:pc,pc('B'))
e10 = set(delim(),:pc,pc('A'))
choral = [e10,e9,e8,e7,e6,e5,e4,e3,e2,e1]

10-element Vector{ChakraImp.Obj}:
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('A')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('B')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('A')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('G')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('G')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('A')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('B')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('D')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('G')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('G')))

# Define viewpoint
The following defines an atomic viewpoint which projects out the pitch class of the last constituent of a sequence

In [5]:
using Viewpoints
v = AtomicViewpoint(pc,:pc)

AtomicViewpoint{pc}(pc, :pc)

### vp_type gives the type of values of the viewpoint:

In [6]:
vp_type(v)

pc

### vp_apply applies the viewpoint to a sequence:

In [7]:
vp_apply(v,choral)

pc('A')

### vp_map turns a sequence of constituents in to a sequence of viewpoint values:

In [8]:
vp_map(v,choral)

10-element Vector{pc}:
 pc('A')
 pc('B')
 pc('A')
 pc('G')
 pc('G')
 pc('A')
 pc('B')
 pc('D')
 pc('G')
 pc('G')

### Viewpoints can be delayed, linked and derived by composition with arbitrary functions:

In [9]:
vp_map(LinkedViewpoint(DelayedViewpoint(v,1),v),choral)

9-element Vector{Tuple{pc, pc}}:
 (pc('B'), pc('A'))
 (pc('A'), pc('B'))
 (pc('G'), pc('A'))
 (pc('G'), pc('G'))
 (pc('A'), pc('G'))
 (pc('B'), pc('A'))
 (pc('D'), pc('B'))
 (pc('G'), pc('D'))
 (pc('G'), pc('G'))

# Create Model 
The following creates an HGramModel of the choral using the pitch class viewpoint. 

In [10]:
using Models
m = Models.HGramModel{pc,3}(vp_map(v,choral))

HGramModel{pc, 3}(Dict{Vector{pc}, Int64}([pc('G'), pc('A')] => 1, [pc('A'), pc('B'), pc('A')] => 1, [pc('G')] => 4, [pc('A'), pc('G')] => 1, [pc('G'), pc('G'), pc('A')] => 1, [pc('A'), pc('B')] => 2, [pc('A'), pc('B'), pc('D')] => 1, [pc('D'), pc('G'), pc('G')] => 1, [pc('G'), pc('A'), pc('B')] => 1, [pc('B'), pc('D')] => 1…), Set(pc[pc('D'), pc('B'), pc('G'), pc('A')]))

### The model elements is a set containing the pitch classes encountered: 

In [11]:
m.elems

Set{pc} with 4 elements:
  pc('D')
  pc('B')
  pc('G')
  pc('A')

### The model database maps sequences to their number of occurences:

In [12]:
m.db

Dict{Vector{pc}, Int64} with 19 entries:
  [pc('G'), pc('A')]          => 1
  [pc('A'), pc('B'), pc('A')] => 1
  [pc('G')]                   => 4
  [pc('A'), pc('G')]          => 1
  [pc('G'), pc('G'), pc('A')] => 1
  [pc('A'), pc('B')]          => 2
  [pc('A'), pc('B'), pc('D')] => 1
  [pc('D'), pc('G'), pc('G')] => 1
  [pc('G'), pc('A'), pc('B')] => 1
  [pc('B'), pc('D')]          => 1
  [pc('B'), pc('D'), pc('G')] => 1
  [pc('B')]                   => 2
  [pc('D'), pc('G')]          => 1
  [pc('A'), pc('G'), pc('G')] => 1
  [pc('A')]                   => 3
  [pc('G'), pc('G')]          => 2
  [pc('B'), pc('A'), pc('G')] => 1
  [pc('D')]                   => 1
  [pc('B'), pc('A')]          => 1

# Maximum likelihood
The function ml computes the maximum likelihood of an element occuring in a given contexts from the model.

In [13]:
ml(pc('G'),m)

0.4

In [14]:
ml(pc('E'),m)

0.0

In [15]:
ml(pc('G'),[pc('G')],m)

0.5

In [16]:
ml(pc('D'),[pc('B'),pc('A')],m)

0.0

# Prediction by partial match
The function ppm blends maximum likelihoods from different order ngram models using interpolated smoothing

In [17]:
ppm(pc('E'),[pc('G')],m)

0.05

# NEXT
- Implement proper smoothing
- How to implement smoothing when alphabet is unknown?
- Long term and short term models